In [1]:
import time
import copy
import random
import pandas as pd
import ipywidgets as widgets
import matplotlib.pyplot as plt
from functools import reduce
from IPython.display import clear_output

In [2]:
class Backtester:
    def __init__(self):
        self.wins = 0
        self.losses = 0
        self.winloss = []
        self.tradelist = []
        
        self.curr_wins = 0
        self.curr_losses = 0
        self.win_streak = 0
        self.loss_streak = 0
    
    def __str__(self):
        return (
            f"Wins: {self.wins}\n"
            f"Losses: {self.losses}\n"
            f"Profit: {self.profit:5.2f}R\n"
            f"Winrate: {self.winrate:5.2f}%\n\n"
            f"Win Streak: {self.win_streak}\n"
            f"Loss Streak: {self.loss_streak}"
        )
    
    def _calc_streaks(self, result: str):
        self.winloss.append(result)
        match result:
            case "w":
                self.curr_wins += 1
                self.curr_losses = 0
            case "l":
                self.curr_losses += 1
                self.curr_wins = 0

        if self.curr_wins > self.win_streak:
            self.win_streak = self.curr_wins
        if self.curr_losses > self.loss_streak:
            self.loss_streak = self.curr_losses
    
    def add_win(self, r):
        self.wins += 1
        self.tradelist.append(r)
        self._calc_streaks("w")        
    
    def add_loss(self, r):
        self.losses += 1
        self.tradelist.append(r)
        self._calc_streaks("l")
    
    @property
    def trades(self):
        return self.wins + self.losses
    
    @property
    def winrate(self):
        if self.trades > 0:
            return (self.wins / (self.trades)) * 100
        else:
            return 0
    
    @property
    def profit(self):
        if len(self.tradelist):
            return reduce(lambda x, y: x + y, self.tradelist)
        else:
            return 0

In [3]:
def calc_risk(risk, amount):
    return round((amount/100.0) * risk)

def equity_curve(starting_balance, risk, trades):
    balance = [starting_balance]
    
    for trade in trades:
        new_balance = balance[-1] + (calc_risk(risk, balance[-1]) * trade)
        balance.append(new_balance)
    
    return balance

class MonteCarlo:
    def __init__(self, balance, risk):
        self.starting_balance = balance
        self.risk = risk
    
    def shuffle(self, runs, data):
        balances = []
    
        for run in range(runs):
            trades = random.sample(data, k=len(data))
            equity = equity_curve(self.starting_balance, self.risk, trades)
            balances.append(equity)
        return balances
    
    def resample(self, runs, data):
        balances = []

        for run in range(runs):
            trades = random.choices(data, k=len(data))
            equity = equity_curve(self.starting_balance, self.risk, trades)
            balances.append(equity)
        return balances

In [ ]:
bt = Backtester()

winbutton = widgets.Button(description="Win")
lossbutton = widgets.Button(description="Loss")
breakevenbutton = widgets.Button(description="Breakeven")
resetbutton = widgets.Button(description="Reset")

winr = widgets.BoundedFloatText(
    value = 1.0,
    min = 0.0,
    max = 10.0,
    step = 0.1,
    description = "Win R: ",
    disabled = False)
lossr = widgets.BoundedFloatText(
    value = -1.0,
    min = -10.0,
    max = 0.0,
    step = 0.1,
    description = "Loss R: ",
    disabled = False)
breakevenr = widgets.BoundedFloatText(
    value = 0.3,
    min = 0.0,
    max = 1.0,
    step = 0.1,
    description = "BE R: ",
    disabled = False)
out = widgets.Output()

def win_click(_):
    bt.add_win(winr.value)
    with out:
        clear_output(wait=True)
        print(bt)
    
def loss_click(_):
    bt.add_loss(lossr.value)
    with out:
        clear_output(wait=True)
        print(bt)
        
def breakeven_click(_):
    bt.add_win(breakevenr.value)
    with out:
        clear_output(wait=True)
        print(bt)
        
def reset_click(_):
    global bt
    bt = Backtester()
    winr.value = 1.0
    lossr.value = -1.0
    breakevenr.value = 0.3
    with out:
        clear_output(wait=True)
        print(bt)

winbutton.on_click(win_click)
lossbutton.on_click(loss_click)
resetbutton.on_click(reset_click)
breakevenbutton.on_click(breakeven_click)

buttons = widgets.HBox([winbutton, lossbutton])
mainbar = widgets.VBox([winr, lossr, breakevenr, buttons, out])
sidebar = widgets.VBox([breakevenbutton, resetbutton])
widgets.HBox([mainbar, sidebar])

In [ ]:
balance = 10000
risk = 2

num_trades = range(bt.trades + 1)
expectancy = (calc_risk(balance, risk) * bt.profit) / bt.trades
equity = equity_curve(balance, risk, bt.tradelist)


fig, ax = plt.subplots()
ax.set_title("Equity Curve")
ax.set_xlabel("Number of Trades")
ax.set_ylabel("Balance in $")
ax.set_xlim([0, bt.trades])

ax.plot(num_trades, equity)
print(f"Start Balance: ${round(balance)}\nEnd Balance: ${round(equity[-1])}\nExpectancy: ${expectancy:5.2f} / Trade")

In [ ]:
sim = MonteCarlo(balance, risk)
sim_data = copy.copy(bt.tradelist)
num_trades = range(bt.trades + 1)

#--------------------------------------------

shuffle = sim.shuffle(1000, sim_data)

shuffle_fig, shuffle_ax = plt.subplots()

shuffle_ax.set_title("Monte Carlo: Shuffle Method")
shuffle_ax.set_xlabel("Number of Trades")
shuffle_ax.set_ylabel("Balance in $")
shuffle_ax.set_xlim([0, bt.trades])

for run in shuffle:
    shuffle_ax.plot(num_trades, run)

#--------------------------------------------    

resample = sim.resample(1000, sim_data)

resample_fig, resample_ax = plt.subplots()

resample_ax.set_title("Monte Carlo: Resample Method")
resample_ax.set_xlabel("Number of Trades")
resample_ax.set_ylabel("Balance in $")
resample_ax.set_xlim([0, bt.trades])

for run in resample:
    resample_ax.plot(num_trades, run)